In [1]:
import json, sys, settings, random
import lasair

/Users/rwillia5/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# the lasair client will be used for pulling all the info about the object
# and for annotating it
L = lasair.lasair_client(settings.API_TOKEN)

In [3]:
selected = 'objects.objectId, sherlock_classifications.classification as class'
tables = 'objects,sherlock_classifications'
conditions = 'ncandgp>20 AND objects.jdmax > %.1f' % (settings.MJD - settings.NEW_TIME)
results = L.query(selected, tables, conditions, limit=settings.MAX_ALERT)

In [4]:
from scipy.stats import ttest_ind

In [5]:
def change_of_mean(alert, old_time, new_time):
    sources  = [c for c in alert['candidates'] if 'candid' in c]
    sources.sort(key = lambda c: c['mjd'])
 
    tlast = sources[-1]['mjd']
    magsold = []
    magsnew = []
    for c in sources:
        if c['mjd'] > tlast - new_time - old_time:
#            print(c['mjd'], c['magpsf'])
            if c['mjd'] > tlast - new_time:
                magsnew.append(c['magpsf'])
            else:
                magsold.append(c['magpsf'])
#    print(objectId, len(sources), len(magsold), len(magsnew))
    if len(magsnew) < 3 or len(magsold) < 3:
        return None
    magsoldmean = sum(magsold)/len(magsold)
    magsnewmean = sum(magsnew)/len(magsnew)

    s = ttest_ind(magsold, magsnew)
    return (s.pvalue, len(magsold), len(magsnew), magsnewmean-magsoldmean)

In [6]:
for row in results:
    objectId = row['objectId']
    alert = L.objects([objectId])[0]
    ret = change_of_mean(alert, settings.OLD_TIME, settings.NEW_TIME)
    if ret: 
        (pvalue, n_old, n_new, new_minus_old) = ret
        if pvalue < 0.05:
            print('https://lasair-ztf.lsst.ac.uk/objects/%s %3s %8.3f (%d %d %.1f)' % \
                  (objectId, row['class'], pvalue, n_old, n_new, new_minus_old))

https://lasair-ztf.lsst.ac.uk/objects/ZTF17aaagnlf  VS    0.008 (17 12 -1.6)
https://lasair-ztf.lsst.ac.uk/objects/ZTF17aaaikoz  CV    0.047 (20 15 0.5)
https://lasair-ztf.lsst.ac.uk/objects/ZTF17aaarsle  VS    0.046 (20 16 -0.8)
https://lasair-ztf.lsst.ac.uk/objects/ZTF18aalphqo  VS    0.000 (10 19 0.8)
https://lasair-ztf.lsst.ac.uk/objects/ZTF18aatpicc  VS    0.003 (11 47 0.4)
